In [ ]:
# MailSense — AI Email Summarizer & Action Planner


In [7]:
import sys
from pathlib import Path

PROJECT_ROOT = Path("..").resolve()
if str(PROJECT_ROOT) not in sys.path:
    sys.path.insert(0, str(PROJECT_ROOT))

print("Project root added to sys.path:", PROJECT_ROOT)


Project root added to sys.path: C:\Users\91759\Desktop\mail-sense


## Demo: Run refined agent pipeline

This demo runs the refined MailSense pipeline (extractor → refined planner → tools) on our example emails.
It demonstrates the following course concepts:
- **Tool Use**: create_task, schedule_event, compose_reply are called as tools.
- **Memory**: recent emails and task index are recorded in `memory.json`.
- **Planning**: `planner_refined` chooses actions and handles conflicts & priorities.

In [8]:
# Notebook cell: run refined agent and display results
from pathlib import Path
import json
from src.agent_main_refined import run_batch

results = run_batch()
print("Agent produced reports for", len(results), "emails.")
# print summaries in a readable form
for r in results:
    print("="*60)
    print("Email subject:", r["email"]["subject"])
    print("Summary:", r["email"]["summary"])
    for p in r["plans"]:
        print(" - Recommendation:", p["recommendation"], "| confidence:", p["confidence"], "| reason:", p["reason"])
        print("   Tool result:", p.get("tool_result"))


Agent produced reports for 6 emails.
Email subject: Meeting request
Summary: Meeting request
 - Recommendation: compose_reply | confidence: 0.6 | reason: Conflict detected with existing scheduled item
   Tool result: {'status': 'ok', 'reply_text': 'Hi there, thanks — could you confirm: which of these slots works for you?'}
Email subject: Invoice #1234 - due Nov 30
Summary: Invoice #1234 - due Nov 30
 - Recommendation: create_task | confidence: 0.97 | reason: Invoice/amount detected: ['$150']
   Tool result: {'status': 'ok', 'task_id': 't-87a8d39a', 'message': 'Task created'}
Email subject: Weekly team update
Summary: Weekly team update. Team — here are the updates for this week: we finished feature X, started Y, and plan Z next week.
 - Recommendation: ignore | confidence: 0.55 | reason: No clear action
   Tool result: None
Email subject: Quick follow-up
Summary: Quick follow-up. Can you send the draft by Monday? Thanks.
 - Recommendation: create_task | confidence: 0.8 | reason: Generi

### Persisted tasks

Show content of `tasks.json` (tasks created by the agent).


In [9]:
from pathlib import Path
import json
p = Path("../tasks.json")  # adjust path if notebook root differs; use correct relative path
if p.exists():
    print(p.read_text())
else:
    print("No tasks.json found yet.")


{
  "tasks": [
    {
      "task_id": "t-dbf1f1f9",
      "title": "Schedule: Meeting request",
      "description": "Hi Arshwin â€” are you free Friday 3â€“4pm for a haircut? If not, next week Tue afternoon works. â€” Priya",
      "created_at": "2025-11-20T14:05:09.395071",
      "due": "2026-03-20T16:00:00",
      "status": "pending",
      "source_email_id": "e-21cff4ea",
      "priority": "medium",
      "tags": [
        "schedule"
      ]
    },
    {
      "task_id": "t-f279a265",
      "title": "Invoice Action: Invoice #1234 - due Nov 30",
      "description": "Dear Arshwin, your invoice #1234 for $150 is due on Nov 30. Please confirm payment.",
      "created_at": "2025-11-20T14:05:09.631369",
      "due": "1234-11-20T00:00:00",
      "status": "pending",
      "source_email_id": "e-527944b5",
      "priority": "high",
      "tags": [
        "invoice"
      ]
    },
    {
      "task_id": "t-f7f6dfe8",
      "title": "Task: Quick follow-up",
      "description": "Can you sen

## Where each course concept is implemented

- **Tool Use (Day 2)**: implemented through functions in `src/tools.py` and invoked by planner/tool dispatcher in `src/agent_main_refined.py`. Tools: `create_task`, `schedule_event`, `compose_reply`.
- **Memory (Day 3)**: persistent memory stored in `memory.json` via `src/memory.py` (functions `add_recent_email`, `add_task_index`).
- **Planning (Day 1 & 5)**: `src/planner_refined.py` implements decision rules, conflict detection, and priority inference.
